In [1]:
import os
import sys
import numpy as np

from PIL import Image
from skimage import morphology,filters

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.layers import Input,Conv2D,Dense,MaxPooling2D,Softmax,Activation,BatchNormalization,\
    Flatten,Dropout,DepthwiseConv2D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
NUM = 14

In [3]:
url = "https://file.liux.pro/d/public/ai-arithmetic/mnist14.tar.gz"

path_to_zip = tf.keras.utils.get_file(origin=url,
                                     cache_dir=".",
                                     cache_subdir="shujuji",
                                     extract=True)

shujuji_dir = os.path.join(os.path.dirname(path_to_zip),'mnist14')

print(shujuji_dir)

.\shujuji\mnist14


In [4]:
data_folder = shujuji_dir

x_data = []
y_data = []
x_data_original = []

def thin(image):
    erzhi = image > filters.threshold_otsu(image)
    gujia = morphology.skeletonize(erzhi)
    thin = (gujia * 255).astype(np.uint8)
    return thin

for label in range(NUM):
    label_folder = os.path.join(data_folder,str(label))
    images = []
    original_images = []
    
    for image_name in os.listdir(label_folder):
        image_path = os.path.join(label_folder,image_name)

        image = Image.open(image_path).convert('L')
        image = image.resize((28, 28))

        image_array = np.array(image)

        original_images.append(image_array)

        image_array = thin(image_array)

        images.append(image_array)
    
    x_data.extend(images)
    y_data.extend([label] * len(images))
    x_data_original.extend(original_images)

x_data = np.array(x_data)
y_data = np.array(y_data)
x_data_original = np.array(x_data_original)

x_use = x_data
y_use = y_data

x_train,x_test,y_train,y_test = train_test_split(x_use, y_use, test_size=0.2, random_state=42)

x_train = x_train.reshape(-1,28,28,1).astype('float32')
x_test = x_test.reshape(-1,28,28,1).astype('float32')

datagen = ImageDataGenerator(
    rotation_range = 1,
    zoom_range = (0.7, 1.0),
    height_shift_range = 0.05
)

datagen.fit(x_train)

unique, counts = np.unique(y_use,return_counts=True)
print("各种图片数量：")
for label,count in zip(unique, counts):
    print(f'类别{label}:{count}张')

各种图片数量：
类别0:11875张
类别1:13533张
类别2:11984张
类别3:12215张
类别4:11622张
类别5:10875张
类别6:11875张
类别7:12496张
类别8:11741张
类别9:11784张
类别10:5059张
类别11:6500张
类别12:5102张
类别13:4234张


In [5]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1) / 255
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1) / 255

y_train = to_categorical(y_train, num_classes=NUM)
y_test = to_categorical(y_test, num_classes=NUM)

In [6]:
def init_model():
    model = Sequential()

    model.add(Conv2D(4, (5, 5), padding='same', strides=(2, 2), input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    model.add(Conv2D(4, (5, 5), padding='same', strides=(3, 3)))
    model.add(BatchNormalization()) 
    model.add(Activation('relu')) 

    model.add(Flatten())

    model.add(Dense(NUM))
    model.add(Dropout(0.3))
    model.add(Activation('softmax'))

    return model


model = init_model()
model.summary()

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["categorical_accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 4)         104       
                                                                 
 batch_normalization (Batch  (None, 14, 14, 4)         16        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 14, 14, 4)         0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 5, 4)           404       
                                                                 
 batch_normalization_1 (Bat  (None, 5, 5, 4)           16        
 chNormalization)                                       

In [ ]:
train_generator = datagen.flow(x_train, y_train, batch_size=128)

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

history = None

if sys.platform.startswith('win'):
    history = model.fit(train_generator,
                        batch_size=1280,
                        epochs=30,
                        verbose=2,
                        validation_data=(x_test, y_test),
                        callbacks=[early_stopping, model_checkpoint],
                        shuffle=True)
else:
    history = model.fit(train_generator,
                        batch_size=1280,
                        epochs=30,
                        verbose=2,
                        validation_data=(x_test, y_test),
                        callbacks=[early_stopping, model_checkpoint],
                        shuffle=True,
                        max_queue_size=100,
                        workers = 8,
                        use_multiprocessing=True)


model.load_weights('best_model.h5')

Epoch 1/30
881/881 - 19s - loss: 1.4709 - categorical_accuracy: 0.5288 - val_loss: 0.7186 - val_categorical_accuracy: 0.8320 - 19s/epoch - 21ms/step
Epoch 2/30


c:\Users\Gaoguiquan\.conda\envs\ai8051u\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


881/881 - 18s - loss: 0.9604 - categorical_accuracy: 0.6751 - val_loss: 0.4817 - val_categorical_accuracy: 0.8856 - 18s/epoch - 21ms/step
Epoch 3/30
881/881 - 18s - loss: 0.8759 - categorical_accuracy: 0.6949 - val_loss: 0.3988 - val_categorical_accuracy: 0.9150 - 18s/epoch - 21ms/step
Epoch 4/30
881/881 - 18s - loss: 0.8372 - categorical_accuracy: 0.7041 - val_loss: 0.3360 - val_categorical_accuracy: 0.9198 - 18s/epoch - 21ms/step
Epoch 5/30
881/881 - 18s - loss: 0.8178 - categorical_accuracy: 0.7089 - val_loss: 0.3166 - val_categorical_accuracy: 0.9197 - 18s/epoch - 20ms/step
Epoch 6/30
881/881 - 18s - loss: 0.8014 - categorical_accuracy: 0.7111 - val_loss: 0.2981 - val_categorical_accuracy: 0.9276 - 18s/epoch - 21ms/step
Epoch 7/30
881/881 - 17s - loss: 0.7853 - categorical_accuracy: 0.7164 - val_loss: 0.2820 - val_categorical_accuracy: 0.9311 - 17s/epoch - 20ms/step
Epoch 8/30
881/881 - 18s - loss: 0.7852 - categorical_accuracy: 0.7153 - val_loss: 0.2725 - val_categorical_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'\n测试集的总体准确率: {test_acc:.2f}')


y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


error_rates = {}
for label in range(NUM):
    label_indices = np.where(y_test_classes == label)[0]
    label_correct = np.sum(y_pred_classes[label_indices] == y_test_classes[label_indices])
    label_total = len(label_indices)
    label_error_rate = 1 - (label_correct / label_total)
    error_rates[label] = label_error_rate

print("\n每种类别的错误率:")
for label, error_rate in error_rates.items():
    print(f'类别 {label}: {error_rate:.2f}')

881/881 [==============================] - 1s 1ms/step - loss: 0.2268 - categorical_accuracy: 0.9402

测试集的总体准确率: 0.94
881/881 [==============================] - 1s 1ms/step

每种类别的错误率:
类别 0: 0.03
类别 1: 0.02
类别 2: 0.07
类别 3: 0.10
类别 4: 0.08
类别 5: 0.09
类别 6: 0.04
类别 7: 0.07
类别 8: 0.09
类别 9: 0.06
类别 10: 0.09
类别 11: 0.00
类别 12: 0.05
类别 13: 0.03


In [10]:
model.export("ending")

!python tools/h5_to_tflite.py ending ending.tflite 1 shujuji/mnist14  0to1
!python tools/tflite2tmdl.py ending.tflite ending.tmdl int8 1 28,28,1 {NUM} 1

INFO:tensorflow:Assets written to: ending\assets


INFO:tensorflow:Assets written to: ending\assets


Saved artifact at 'ending'. The following endpoints are available:

* Endpoint 'serve'
  Args:
    args_0: float32 Tensor, shape=(None, 28, 28, 1)
  Returns:
    float32 Tensor, shape=(None, 14)
Done, quant used time:1.1540379524230957


2025-03-06 20:07:54.220563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use `tf.saved_model.load` instead.
2025-03-06 20:07:54.241034: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2025-03-06 20:07:54.326794: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-03-06 20:07:54.327087: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions

CONV_2D
{'dilation_h_factor': 1, 'dilation_w_factor': 1, 'fused_activation_function': 1, 'padding': 0, 'stride_h': 2, 'stride_w': 2}
    input: serving_default_conv2d_input
    output: sequential/activation/Relu;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/activation/Relu;sequential/batch_normalization/FusedBatchNormV3;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/batch_normalization/FusedBatchNormV3;conv2d/bias;sequential/conv2d/BiasAdd;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/conv2d/BiasAdd;sequential/conv2d_1/Conv2D;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/conv2d_1/Conv2D;sequential/conv2d/Conv2D;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/conv2d/Conv2D1
    filter 7: sequential/conv2d/Conv2D;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/conv2d/Conv2D 
    bias 6: sequential/activation/Relu;StatefulPartitionedCall_1/StatefulPartitionedCall/sequential/activation/Relu;sequential/batc

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
